In [2]:
import requests
import json

# URL-id
notes_url = "https://github.com/andylolz/community-notes-explorer/raw/gh-pages/data/notes.json"
meta_url = "https://github.com/andylolz/community-notes-explorer/raw/gh-pages/_data/meta.json"

# Failide nimed
notes_file = "notes.json"
meta_file = "meta.json"

# Lae alla notes.json
notes_response = requests.get(notes_url)
with open(notes_file, "wb") as f:
    f.write(notes_response.content)

# Lae alla meta.json
meta_response = requests.get(meta_url)
with open(meta_file, "wb") as f:
    f.write(meta_response.content)

# Lae andmed mälu sisse
with open(notes_file, "r", encoding="utf-8") as f:
    notes = json.load(f)

with open(meta_file, "r", encoding="utf-8") as f:
    meta = json.load(f)

# Väljasta esimesed 3 Community Note’i ja nende seotud tweetid
print("📝 Community Notes (esimesed 3):\n")

for i, note in enumerate(notes[:3]):
    tweet_url = f"https://twitter.com/i/web/status/{note['tweetId']}"
    print(f"🔹 {i+1}. Säutsu link: {tweet_url}")
    print(f"📌 Märkus: {note.get('summary', '[Puudub kokkuvõte]')}")
    print(f"👥 Märkuse autor: {note.get('participantId')}")
    print(f"📅 Loodud: {note.get('createdAt')}")
    print(f"—" * 50)


📝 Community Notes (esimesed 3):



KeyError: 'tweetId'

In [8]:
import json
import pandas as pd

# === 1. Lae notes.json ===
with open("notes.json", "r", encoding="utf-8") as f:
    notes_data = json.load(f)

notes_df = pd.DataFrame(notes_data)
notes_df["tweet_id"] = notes_df["tweet_id"].astype(str)
print(f"[1] notes_df: {notes_df.shape[0]} rida")

# === 2. Lae Twitter_New_Dataset.json ===
with open("Twitter_New_Dataset.json", "r", encoding="utf-8") as f:
    twitter_data = json.load(f)

twitter_df = pd.json_normalize(twitter_data)
twitter_df["tweet_id"] = twitter_df["id"].astype(str)
print(f"[2] twitter_df: {twitter_df.shape[0]} rida")

# === 3. Liida notes_df ja twitter_df säutsu ID alusel ===
merged_1 = pd.merge(notes_df, twitter_df, on="tweet_id", how="inner")
print(f"[3] Pärast merge notes + twitter: {merged_1.shape[0]} rida")

# === 4. Lae engagement CSV ja puhasta tweet_id (eemalda 'tw-' eesliide) ===
engagement_df = pd.read_csv("Tweets_and_User_Engagement.csv", dtype=str)
engagement_df["tweet_id"] = engagement_df["TweetID"].str.replace("tw-", "", regex=False)
print(f"[4] engagement_df: {engagement_df.shape[0]} rida")

# === 5. Liida eelmise tulemusega ka engagement_df ===
final_merged = pd.merge(merged_1, engagement_df, on="tweet_id", how="inner")
print(f"[5] Pärast merge eelmise + engagement: {final_merged.shape[0]} rida")

# === 6. Salvesta lõplik andmestik uude faili ===
final_merged.to_csv("merged_notes_data.csv", index=False, encoding="utf-8")
print("[6] Fail salvestatud: merged_notes_data.csv")


print(notes_df["tweet_id"].head())
print(twitter_df["tweet_id"].head())
print(engagement_df["tweet_id"].head())

[1] notes_df: 5767 rida
[2] twitter_df: 4608 rida
[3] Pärast merge notes + twitter: 0 rida
[4] engagement_df: 206295 rida
[5] Pärast merge eelmise + engagement: 0 rida
[6] Fail salvestatud: merged_notes_data.csv
0    1929694788510798080
1    1929632247092859247
2    1929576973874999696
3    1929563391854522661
4    1929513596167336253
Name: tweet_id, dtype: object
0    1762265250987200925
1    1762265102085165546
2    1762264840154857577
3    1762264888053821624
4    1762264739139510735
Name: tweet_id, dtype: object
0    682712873332805633
1    682713045357998080
2    682713219375476736
3    682713436967579648
4    682714048199311366
Name: tweet_id, dtype: object


In [10]:
print("Kattuvad IDsid notes_df ja twitter_df vahel:")
print(set(notes_df['tweet_id']).intersection(set(twitter_df['tweet_id'])))

print("Kattuvad IDsid notes_df ja engagement_df vahel:")
print(set(notes_df['tweet_id']).intersection(set(engagement_df['tweet_id'])))

print("Kattuvad IDsid twitter_df ja engagement_df vahel:")
print(set(twitter_df['tweet_id']).intersection(set(engagement_df['tweet_id'])))

print("Notes_df tweet_id näited:", notes_df['tweet_id'].head(10).tolist())
print("Twitter_df tweet_id näited:", twitter_df['tweet_id'].head(10).tolist())
print("Engagement_df tweet_id näited:", engagement_df['tweet_id'].head(10).tolist())


Kattuvad IDsid notes_df ja twitter_df vahel:
set()
Kattuvad IDsid notes_df ja engagement_df vahel:
set()
Kattuvad IDsid twitter_df ja engagement_df vahel:
set()
Notes_df tweet_id näited: ['1929694788510798080', '1929632247092859247', '1929576973874999696', '1929563391854522661', '1929513596167336253', '1929608526160077139', '1929489015234871684', '1929517357019381943', '1929529534488977667', '1929671962843865446']
Twitter_df tweet_id näited: ['1762265250987200925', '1762265102085165546', '1762264840154857577', '1762264888053821624', '1762264739139510735', '1762264372179616209', '1762264205397238201', '1762264135511797881', '1762263858713108905', '1762263752462786845']
Engagement_df tweet_id näited: ['682712873332805633', '682713045357998080', '682713219375476736', '682713436967579648', '682714048199311366', '682714583044243456', '682715526435454977', '682717209043402753', '682717967965110273', '682718048080519168']


In [13]:
import json
import pandas as pd

# 1. Lae notes.json ja prindi esimesed read
with open("notes.json", "r", encoding="utf-8") as f:
    notes_data = json.load(f)
print("notes.json esimese 2 kirjet:")
for i, item in enumerate(notes_data[:2]):
    print(f"[{i+1}] {item}")
print("\n")

# 2. Lae Twitter_New_Dataset.json ja prindi esimesed read
with open("Twitter_New_Dataset.json", "r", encoding="utf-8") as f:
    twitter_data = json.load(f)
print("Twitter_New_Dataset.json esimese 2 kirjet:")
for i, item in enumerate(twitter_data[:2]):
    print(f"[{i+1}] {item}")
print("\n")

# 3. Lae Tweets_and_User_Engagement.csv ja prindi esimesed read
engagement_df = pd.read_csv("Tweets_and_User_Engagement.csv", dtype=str, nrows=2)
print("Tweets_and_User_Engagement.csv esimese 2 rida:")
print(engagement_df)
print("\n")


notes.json esimese 2 kirjet:
[1] {'dl': 1, 'lang': 'ja', 'tweet': '毎週、自治体がコロナの感染状況を報告してますが、定点観測は誤魔化しです\n１医療機関あたり何人と言われても、全体で何人感染者がいるか全くわかりません\n患者の総数や下水中のウイルス量を調べなければ感染状況はわかりません\n定点観測なんてやってるのは日本と韓国くらいです', 'tweet_created_at': '2025-06-03T00:21:04.000Z', 'tweet_id': '1929694788510798080', 'user': 'robin1918081', 'user_id': '1743360349917843456', 'notes': [{'created_at': '2025-06-03T01:03:36+00:00', 'note_id': '1929705493544718542', 'reasons': [1, 3], 'summary': '誤った情報を含むポストです。    全数把握をせず、定点把握のようにトレンド監視に移行した国や報告を停止した国はいくつもあります。    特に米国は2025年の症例定義では、個別の症例報告からトレンド監視に移行しており、全数把握は行っていません。  <a target="_blank" href="https://ndc.services.cdc.gov/case-definitions/coronavirus-disease-2019-2025/">https://ndc.services.cdc.gov/case-definitio…</a>'}]}
[2] {'dl': 1, 'lang': 'de', 'tweet': 'Haben sich Klimakleber schon am Fuße des #Ätna aus Protest festgeklebt um gegen seinen CO2 Austoß zu protestieren?\nDer Vulkan versaut mit seinem Ausbruch ja schließlich Deutschlands Klimabilanz mit der wir im Alleing

In [ ]:
print(set(twitter_data["id_column"]) & set(df2["id_column"]))

NameError: name 'df1' is not defined